<a href="https://colab.research.google.com/github/Rikupro-creator/Learning-Langchain/blob/main/langchain_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is another langchain tutorial on how to use and how to implement a Retrieval RAG. This one also provides the instances that are similar to our query. The code is much borrowed from this link - https://www.python-engineer.com/posts/langchain-crash-course/

Lets start by installing the necessary packages that we need for this project.

In [ ]:
pip install langchain langchain_community openai tiktoken huggingface_hub faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 4.8 MB/s eta 0:00:00


as usual, lets start with a basic usage of LLMs in Langchain

In [ ]:
import os
os.environ['OPENAI_API_KEY']=''

In [ ]:
from langchain.llms import OpenAI
llm=OpenAI(temperature=.7)
llm('what is the best programming language to learn today?')


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'\n\nThe best programming language to learn today depends on personal goals, interests, and career opportunities. Some popular languages to consider are Python, Java, JavaScript, C++, and Go. It is also important to research the current demand and job market for different languages in your area. Ultimately, the best language to learn is one that aligns with your goals and has a strong community and resources for support and growth.'

# Prompts

lets look at why we need prompts see the code chunks below.

In [ ]:

llm("Can Barack Obama have a conversation with George Washington?")

'\nNo, Barack Obama cannot have a conversation with George Washington as Washington passed away in 1799 and Obama was not born until 1961. They cannot physically meet or communicate with each other.'

In [ ]:
prompt = """Question: Can Barack Obama have a conversation with George Washington?

Let's think step by step.

Answer: """
llm(prompt)

'\nTechnically, no, as George Washington passed away in 1799, long before Barack Obama was born. However, if we were to imagine a hypothetical scenario where time travel or some other form of communication was possible, then yes, it is possible for Barack Obama to have a conversation with George Washington.'

You see the answers. Prompting an LLM is so important to ensure that we get the results that we need.

we can do this using prompt templates which is the easiest to use in  langchain.

In [ ]:
from langchain.prompts import PromptTemplate
template = """Question: {question}

Let's think step by step.

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

now we have created the prompt. lets  include it with the question

In [ ]:
prompt.format(question="Can Barack Obama have a conversation with George Washington?")


"Question: Can Barack Obama have a conversation with George Washington?\n\nLet's think step by step.\n\nAnswer: "

this is how we can ask the llm questions regrading the topic

In [ ]:
# method 1

llm(prompt.format(question="Can Barack Obama have a conversation with George Washington?"))

"No, it is not possible for Barack Obama to have a conversation with George Washington as George Washington passed away in 1799, more than 200 years before Barack Obama's presidency."

In [ ]:
# method 2

from langchain import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Can Barack Obama have a conversation with George Washington?"

print(llm_chain.run(question))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



Technically, no because George Washington passed away in 1799 and Barack Obama was born in 1961. However, if we were to imagine a hypothetical scenario where time travel was possible, then it would be possible for Barack Obama to have a conversation with George Washington.


# Agents and Tools

Agents involve an LLM making decisions about which cctions to take, taking that cction, seeing an observation, and repeating that until done.

When used correctly agents can be extremely powerful. In order to load agents, you should understand the following concepts:

  -Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains. See available Tools.

  -LLM: The language model powering the agent.
  
  -Agent: The agent to use. See also Agent Types.




In [ ]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=9232e73f962c5daf067d46d7c5f900655c5a440a8d08d8db14df40090c448aa2
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [ ]:
from langchain.llms import openai
lm=OpenAI(temperature=0)
tools=load_tools(['wikipedia', 'llm-math'], llm=llm)

In [ ]:
agent=initialize_agent(tools, llm, agent='zero-shot-react-description', verbose=True)

In [ ]:
agent.run('In what year was the film Departed with Leopnardo Dicaprio released? What is this year raised to the 0.43 power?')



> Entering new AgentExecutor chain...
 We can use Wikipedia to find the release year of the film.
Action: wikipedia
Action Input: The Departed

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: The Departed
Summary: The Departed is a 2006 American epic crime thriller film film directed by Martin Scorsese and written by William Monahan. It is both a remake of the 2002 Hong Kong film Infernal Affairs and also loosely based on the real-life Boston Winter Hill Gang; the character Colin Sullivan is based on the corrupt FBI agent John Connolly, while the character Frank Costello is based on Irish-American gangster and crime boss Whitey Bulger. The film stars Leonardo DiCaprio, Matt Damon, Jack Nicholson, and Mark Wahlberg, with Martin Sheen, Ray Winstone, Vera Farmiga, Alec Baldwin, Anthony Anderson and James Badge Dale in supporting roles.
The film takes place in Boston and the surrounding metro area, primarily in the South Boston neighborhood. Irish Mob boss Frank Costello (Nicholson) plants Colin Sullivan (Damon) as a spy within the Massachusetts State Police; simultaneously, the police assign undercover state trooper Billy Costigan (DiCaprio) to infiltrate C

'The final answer is 26.30281917656938.'

# memory

Memory is the concept of persisting state between calls of a chain/agent. LangChain provides a standard interface for memory, a collection of memory implementations, and examples of chains/agents that use memory.

In [ ]:
from langchain import OpenAI, ConversationChain

llm=OpenAI(temperature=0)

conversation=ConversationChain(llm=llm, verbose=True)
conversation.predict(input='Hi there!')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?"

In [ ]:
conversation.predict(input="Can we talk about AI?")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?
Human: Can we talk about AI?
AI:

> Finished chain.


' Absolutely! I am an AI, so I am always happy to talk about my own kind. What specifically would you like to know about AI?'

In [ ]:
conversation.predict(input="I'm interested in Reinforcement Learning.")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?
Human: Can we talk about AI?
AI:  Absolutely! I am an AI, so I am always happy to talk about my own kind. What specifically would you like to know about AI?
Human: I'm interested in Reinforcement Learning.
AI:

> Finished chain.


' Reinforcement Learning is a type of machine learning that involves training an AI agent through trial and error. The agent receives rewards or punishments based on its actions, and uses this feedback to learn how to achieve a specific goal. It is often used in tasks that involve decision making and long-term planning, such as playing games or controlling robots. Is there anything else you would like to know about Reinforcement Learning?'

and this is the end of the tutorial on the basics oflangchain.